In [1]:
import polars_ds as pld
import polars as pl
import numpy as np 

# Num Extensions

In [2]:
size = 100_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "a": np.random.random(size = size)
    , "b": np.random.random(size = size)
    , "x1" : range(size)
    , "x2" : range(size, size + size)
    , "y": range(-size, 0)
    , "actual": np.round(np.random.random(size=100_000)).astype(np.int32)
    , "predicted": np.random.random(size=100_000)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
})
df.head()

f,dummy,a,b,x1,x2,y,actual,predicted,dummy_groups
f64,str,f64,f64,i64,i64,i64,i32,f64,str
0.0,"""a""",0.329054,0.431276,0,100000,-100000,0,0.194481,"""a"""
0.841471,"""a""",0.302734,0.186427,1,100001,-99999,1,0.615612,"""a"""
0.909297,"""a""",0.066187,0.955182,2,100002,-99998,1,0.953673,"""a"""
0.14112,"""a""",0.052694,0.399579,3,100003,-99997,0,0.90706,"""a"""
-0.756802,"""a""",0.618107,0.307307,4,100004,-99996,0,0.115548,"""a"""


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pl.col("x1").num.jaccard(pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT
df.select(
    pl.col("f").num.rfft()
).head()

f
list[f64]
"[1.812028, 0.0]"
"[1.812028, -0.000002]"
"[1.812028, -0.000005]"
"[1.812028, -0.000007]"
"[1.812028, -0.00001]"


In [5]:
# Least Square (Linear Regression)
df.select(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
)

y
list[f64]
"[2.0, -1.0]"


In [6]:
df.lazy().select(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
).collect()

y
list[f64]
"[2.0, -1.0]"


In [7]:
df.select(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False).over(pl.col("dummy"))
).head()

list_float
list[f64]
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"


In [8]:
df.group_by("dummy").agg(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
)


dummy,list_float
str,list[f64]
"""b""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"


In [9]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pl.col("y").num.cond_entropy(pl.col("x1"))
)

y
f64
-0.0


In [10]:
df.group_by("dummy_groups").agg(
    pl.col("actual").metric.l2_loss(pl.col("predicted")).alias("l2"),
    pl.col("actual").metric.bce(pl.col("predicted")).alias("log loss"),
    pl.col("actual").metric.binary_metrics_combo(pl.col("predicted")).alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.335546,1.005173,0.498574,0.496142,0.248677,0.495425,0.495449
"""a""",0.334022,0.997736,0.500377,0.502168,0.250635,0.501932,0.498258


# Str Extension

In [11]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""


In [12]:
# Tokenize
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize().explode().unique()
)

sen
str
"""to"""
"""hello"""
"""going"""
"""world"""
"""church"""


In [13]:
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize(stem=True).explode().unique()
)

sen
str
"""go"""
"""hello"""
"""church"""
"""world"""


In [14]:
df2.select(
    pl.col("word").str2.levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [15]:
# Damerau-Levenshtein
df2.select(
    pl.col("word").str2.d_levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [16]:
df2.select(
    pl.col("word").str2.levenshtein("world", return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [17]:
df2.filter(
    pl.col("word").str2.levenshtein("world") == 1
).head()

sen,word
str,str
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""


In [18]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [19]:
df.filter(
    pl.col("word").str2.similar_to_vocab(
        vocab = gibberish,
        threshold = 0.5,
        metric = "leven", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [20]:

df.select(
    pl.col("word").str2.levenshtein("asasasa", return_sim=True).alias("asasasa"),
    pl.col("word").str2.levenshtein("sasaaasss", return_sim=True).alias("sasaaasss"),
    pl.col("word").str2.levenshtein("asdasadadfa", return_sim=True).alias("asdasadadfa"),
    pl.col("word").str2.fuzz("apples").alias("LCS based Fuzz match - apples"),
    pl.col("word").str2.osa("apples", return_sim = True).alias("Optimal String Alignment - apples"),
    pl.col("word").str2.jw("apples").alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [21]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
0.375437
0.9494
-0.651141


In [22]:
# Genenrate random sample, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pl.col("a").stats.sample_normal(mean = 0.5, std = 1., respect_null=True).alias("random")
)

a,random
f64,f64
null,null
null,null
0.375437,0.771642
0.9494,0.545358
-0.651141,-1.091522
-1.834427,0.610844
-0.620926,1.264071
1.812079,-0.381095
2.110361,-0.321377


In [23]:
# Genenrate random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 1, max_size = 5, respect_null=True).alias("random_str")
)

a,random_str
f64,str
null,null
null,null
0.375437,"""QcAN"""
0.9494,"""9"""
-0.651141,"""wA"""
-1.834427,"""81"""
-0.620926,"""rhXk"""
1.812079,"""KfKa"""
2.110361,"""zs"""


In [24]:
# Genenrate fixed size random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 5, max_size = 5, respect_null=True).alias("random_str")
)

a,random_str
f64,str
null,null
null,null
0.375437,"""G35lQ"""
0.9494,"""m8OqI"""
-0.651141,"""S7CWK"""
-1.834427,"""IgOkR"""
-0.620926,"""NbTmT"""
1.812079,"""Trx1u"""
2.110361,"""VCvz1"""


In [25]:
df.with_columns(
    # Sample from normal distribution, using reference column "a" 's mean and std
    pl.col("a").stats.sample_normal().alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.col("a").stats.sample_uniform(low = 0., high = None, respect_null=True).alias("test2")
).head()

a,test1,test2
f64,f64,f64
null,-0.316657,null
null,0.857924,null
0.375437,0.817816,0.840231
0.9494,-0.145348,1.610263
-0.651141,-1.057825,1.090286


In [26]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pl.col("a").stats.sample_normal(mean = 0.5, std = 1.).alias("test1")
    , pl.col("a").stats.sample_normal(mean = 0.5, std = 2.).alias("test2")
).select(
    pl.col("test1").stats.ttest_ind(pl.col("test2"), equal_var = False).alias("t-test")
    , pl.col("test1").stats.normal_test().alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
-0.340638,0.733424,0.489244,0.783001


In [27]:
size = 5000
df = pl.DataFrame({
    "market_id": range(size),
    "group1": np.random.random(size=size),
    "group2": np.random.random(size=size),
    "category_1": np.random.randint(low=0, high=5, size=size),
    "category_2":np.random.randint(low=0, high=10, size=size)
}).with_columns(
    pl.col("market_id").mod(3)
)
df.head(5)

market_id,group1,group2,category_1,category_2
i64,f64,f64,i64,i64
0,0.279683,0.693541,3,7
1,0.165041,0.408362,3,7
2,0.36538,0.943691,1,5
0,0.182031,0.066145,1,8
1,0.451541,0.953597,3,1


In [28]:
# In dataframe statistical tests!
print(df.select(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = True).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
))

shape: (1, 3)
┌──────────────────────┬──────────────────────┬─────────────────────┐
│ t-test               ┆ chi2-test            ┆ f-test              │
│ ---                  ┆ ---                  ┆ ---                 │
│ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞══════════════════════╪══════════════════════╪═════════════════════╡
│ {-1.650504,0.098871} ┆ {36.895283,0.427337} ┆ {0.309968,0.871476} │
└──────────────────────┴──────────────────────┴─────────────────────┘


In [29]:
# Can also be done in group by context
df.group_by("market_id").agg(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = False).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),-
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
)

market_id,t-test,chi2-test,f-test
i64,struct[2],struct[2],struct[2]
0,"{-1.237199,0.2161}","{32.480736,0.636743}","{-0.373944,-0.82735}"
1,"{0.056369,0.955051}","{42.838918,0.201173}","{-0.29107,-0.883896}"
2,"{-1.668119,0.095386}","{52.244235,0.039157}","{-0.690292,-0.598664}"


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [30]:
import polars_ds as pld
df = pl.DataFrame({
    "id": range(1000),
    "val1": np.random.random(size=1000), 
    "val2": np.random.random(size=1000), 
    "val3": np.random.random(size=1000),
    "r": np.random.random(size=1000),
    "rh": np.random.random(size=1000)*10,
})

In [31]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.select(
    pld.query_nb_cnt(
        0.1, # radius 
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        dist = "inf", # L Infinity distance 
        parallel = True 
    )
).head()

""
u32
10
5
5
10
3


In [32]:
df.select(
    pld.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        dist = "l1", # L 1 distance 
        parallel = True 
    )
).head()

r
u32
117
18
154
355
184


In [33]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pl.col("id").num.knn_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends")
).head()

id,val1,val2,val3,r,rh,best friends
i64,f64,f64,f64,f64,f64,list[u64]
0,0.628277,0.24721,0.823609,0.401848,4.325744,"[0, 629, … 857]"
1,0.359997,0.586155,0.179896,0.06172,4.930872,"[1, 399, … 635]"
2,0.974186,0.90477,0.223832,0.119878,9.149884,"[2, 902, … 53]"
3,0.15599,0.46002,0.756781,0.680912,6.204805,"[3, 391, … 898]"
4,0.254709,0.305821,0.035771,0.770254,7.867602,"[4, 94, … 816]"


In [34]:
# Filter to only points near the given point
df.filter(
    pld.query_radius(
        [0.5, 0.5, 0.5],
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        radius = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,val1,val2,val3,r,rh
i64,f64,f64,f64,f64,f64
0,0.628277,0.24721,0.823609,0.401848,4.325744
1,0.359997,0.586155,0.179896,0.06172,4.930872
3,0.15599,0.46002,0.756781,0.680912,6.204805
6,0.490409,0.513039,0.853946,0.568392,8.889839
8,0.728796,0.803011,0.33399,0.025543,1.634557


In [35]:
# Haversine distance is available when dimension is 2
df.filter(
    pld.query_radius(
        [0.5, 0.5],
        pl.col("val1"), pl.col("val2"), # Columns used as the coordinates in n-d space
        radius = 10, # in km
        dist = "h" 
    )
).head()

id,val1,val2,val3,r,rh
i64,f64,f64,f64,f64,f64
6,0.490409,0.513039,0.853946,0.568392,8.889839
12,0.424308,0.494102,0.532104,0.352054,4.542275
45,0.502154,0.583782,0.14791,0.25895,7.132816
65,0.544417,0.502539,0.229982,0.349951,2.068083
74,0.496812,0.421964,0.761863,0.979747,1.770039


In [36]:
df.filter(
    pld.query_radius(
        [0.5, 0.5],
        # Columns used as the coordinates in n-d space
        pl.col("val1"), pl.col("val2"), 
        # radius can also be an existing column in the dataframe.
        radius = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,val1,val2,val3,r,rh
i64,f64,f64,f64,f64,f64
6,0.490409,0.513039,0.853946,0.568392,8.889839
133,0.498075,0.517661,0.196447,0.756156,9.906723
164,0.496588,0.493433,0.39586,0.202915,1.926145
215,0.490266,0.520783,0.488036,0.521546,5.905562
248,0.469321,0.526669,0.117629,0.563881,9.793812
